In [40]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time

In [41]:
df = pd.read_csv("companies.csv")  
companies = df['company_processed'].dropna().unique().tolist()

In [42]:
def apply_classification_filter():
    try:
        try:
            label = wait.until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "label[for='ClassificationsField__Classifications']")
                )
            )
            label.click()
        except Exception:
            # Fallback: click the checkbox directly via JS
            checkbox = wait.until(
                EC.element_to_be_clickable(
                    (By.ID, "ClassificationsField__Classifications")
                )
            )
            driver.execute_script("arguments[0].click();", checkbox)

        # Wait until the panel is actually visible
        wait.until(
            EC.visibility_of_element_located(
                (By.ID, "classificationsPanel")
            )
        )

        # 2. Click "Information & Communication Technology"
        ict_option = wait.until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "a[data-testid='6281']")
            )
        )
        ict_option.click()

        # 3. Click "Done" to apply the filter
        done_button = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//div[@id='classificationsPanel']//span[text()='Done']")
            )
        )
        done_button.click()

    except Exception as e:
        print(f"Could not apply ICT classification filter: {e}")

In [43]:
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

results = []


for company in companies:
    driver.get("https://www.seek.com.au")
    time.sleep(2)

    try:
        search_box = driver.find_element(By.ID, "keywords-input")
    except:
        # fallback selector in case Seek changes IDs
        search_box = driver.find_element(By.CSS_SELECTOR, "input[type='text']")

    search_box.clear()
    search_box.send_keys(company)
    search_box.send_keys(Keys.RETURN)
    
    # Apply filters
    # apply_classification_filter() 
    
    time.sleep(3)

    job_cards = driver.find_elements(By.CSS_SELECTOR, "a[data-automation='jobTitle']")

    for card in job_cards:
        results.append({
            "company": company,
            "job_title": card.text,
            "job_link": card.get_attribute("href")
        })

    print(f"Finished: {company} ({len(job_cards)} jobs)")  # sanity output

driver.quit()

Finished: sydney night patrol & inquiry co   (22 jobs)
Finished: flying bark productions   (0 jobs)
Finished: employers mutual management   (22 jobs)
Finished: crawford & company australia   (14 jobs)
Finished: isentia   (3 jobs)
Finished: bmw australia  (22 jobs)
Finished: australia and new zealand banking group  (22 jobs)
Finished: fantech   (1 jobs)
Finished: ap facilities   (22 jobs)
Finished: zurich financial services australia  (21 jobs)
Finished: electro optic systems   (22 jobs)
Finished: bendigo and adelaide bank  (22 jobs)
Finished: gn hearing australia   (8 jobs)
Finished: wtg   (7 jobs)
Finished: mecca brands   (13 jobs)
Finished: little company of mary health care  (22 jobs)
Finished: avk australia holding   (4 jobs)
Finished: capgemini australia   (22 jobs)
Finished: ventia australia   (0 jobs)
Finished: ausreo   (7 jobs)
Finished: ecu worldwide australia   (7 jobs)
Finished: nanosonics  (22 jobs)
Finished: seventhday adventist schools victoria  (22 jobs)
Finished: under 

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=142.0.7444.162)
Stacktrace:
0   chromedriver                        0x000000010453faa4 cxxbridge1$str$ptr + 2943380
1   chromedriver                        0x0000000104537760 cxxbridge1$str$ptr + 2909776
2   chromedriver                        0x000000010404e2bc _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74028
3   chromedriver                        0x0000000104037a30 chromedriver + 211504
4   chromedriver                        0x000000010405ae98 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 126216
5   chromedriver                        0x00000001040bde08 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 531576
6   chromedriver                        0x00000001040d68d8 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 63264

In [44]:
output_df = pd.DataFrame(results)
output_df.to_csv("seek_job_results.csv", index=False)

print("\nDone. Saved to seek_job_results.csv")


Done. Saved to seek_job_results.csv
